In [5]:
import os

root_dir = "."   # 프로젝트 최상위
rel_path = os.path.join(
    "CHB-MIT Dataset",
    "Patient Specific",
    "Intra-specific",
    "Seizure Patterns"
)
search_root = os.path.join(root_dir, rel_path)

In [6]:
import re

pattern = re.compile(
    r'chb(\d{2}_\d{2})_(pre-ictal|ictal|post-ictal)\.csv$',
    re.IGNORECASE
)


In [7]:
groups = {}   # { "01_03": { "pre-ictal": path1, "ictal": path2, … }, ... }

for dirpath, _, filenames in os.walk(search_root):
    for fname in filenames:
        m = pattern.match(fname)
        if not m:
            continue
        idx, phase = m.groups()
        groups.setdefault(idx, {})[phase.lower()] = os.path.join(dirpath, fname)


In [ ]:
import pandas as pd

output_dir = os.path.join(search_root, "combined1")
os.makedirs(output_dir, exist_ok=True)

for idx, phases in groups.items():
    dfs = []
    for phase in ("pre-ictal", "ictal", "post-ictal"):
        path = phases.get(phase)
        if not path:
            print(f"⚠️ 누락: {idx} → {phase}")
            continue

        # 헤더 없는 CSV로 읽기
        df = pd.read_csv(path, header=None)
        # 마지막(쓰레기) 행 제거
        df = df.iloc[:-1]
        dfs.append(df)

    if not dfs:
        continue

    # 그냥 세 데이터프레임을 세로로 붙임
    combined = pd.concat(dfs, ignore_index=True)

    out_path = os.path.join(output_dir, f"chb_{idx}_seizure.csv")
    # 결과물도 헤더 없이 저장
    combined.to_csv(out_path, index=False, header=False)
    print(f"✅ 저장됨: {out_path}")


✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_03_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_04_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_15_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_16_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_18_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_21_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_01_26_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_21_19_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Seizure Patterns\combined1\chb_21_20_seizure.csv
✅ 저장됨: .\CHB-MIT Dataset\Patient Specific\Intr

In [12]:
import os
import re
import pandas as pd

def process_non_seizure_files(root_dir: str, output_dir: str = None):
    """
    root_dir 이하를 순회하며, 지정된 chbXX_YY ID 목록에 해당하는
    'non-seizure' CSV 파일을 찾아 마지막 행을 제거한 뒤
    combined_non_seizure 폴더에 저장합니다.
    """
    # 처리할 파일 ID 목록
    desired_ids = {
        "chb01_01", "chb01_05", "chb01_07", "chb01_09", "chb01_11",
        "chb01_13", "chb01_17", "chb21_01", "chb21_02", "chb21_03", "chb21_04"
    }

    # Non-Seizure Patterns 하위 경로
    rel_path = os.path.join(
        "CHB-MIT Dataset",
        "Patient Specific",
        "Intra-specific",
        "Non-Seizure Patterns"
    )
    search_root = os.path.join(root_dir, rel_path)

    # 출력 폴더 설정
    if output_dir is None:
        output_dir = os.path.join(search_root, "combined_non_seizure")
    os.makedirs(output_dir, exist_ok=True)

    # 파일명에서 ID를 뽑아낼 정규식
    pattern = re.compile(r'^(chb\d{2}_\d{2})_non-seizure.*\.csv$', re.IGNORECASE)

    for dirpath, _, filenames in os.walk(search_root):
        for fname in filenames:
            m = pattern.match(fname)
            if not m:
                continue
            file_id = m.group(1)
            if file_id not in desired_ids:
                continue

            src = os.path.join(dirpath, fname)
            # --- 여기가 핵심: pandas로 읽어서 마지막 행 제거 ---
            df = pd.read_csv(src, header=None)   # 헤더 없는 CSV로 읽기
            df = df.iloc[:-1]                    # 마지막 행(쓰레기) 제거

            # 저장 파일명: chbXX_YY_<원본파일명>.csv
            dst_name = f"{fname}"
            dst = os.path.join(output_dir, dst_name)
            df.to_csv(dst, index=False, header=False)
            print(f"✅ Processed & Saved: {dst}")

if __name__ == "__main__":
    # 프로젝트 최상위 폴더를 지정하기만 하면 됩니다.
    process_non_seizure_files(root_dir=".")


✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_01_non-seizure(ictal).csv
✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_05_non-seizure(ictal).csv
✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_07_non-seizure(ictal).csv
✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_09_non-seizure(ictal).csv
✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_11_non-seizure(ictal).csv
✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_13_non-seizure(ictal).csv
✅ Processed & Saved: .\CHB-MIT Dataset\Patient Specific\Intra-specific\Non-Seizure Patterns\combined_non_seizure\chb01_17_non-seiz